In [ ]:
!pip install nb_black > /dev/null

In [ ]:
import json
from os.path import basename

from collections import defaultdict, Counter
from glob import glob

import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MultiLabelBinarizer

plt.style.use("ggplot")

%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
def split(s, sep):
    return s.split(sep) if s != "" else []

In [ ]:
train_soundscape_labels = pd.read_csv(
    "../input/birdclef-2021/train_soundscape_labels.csv"
)
train_soundscape_labels["recording_id"] = train_soundscape_labels.row_id.apply(
    lambda x: int(x.split("_")[0])
)
train_soundscape_labels["end_sec"] = train_soundscape_labels.row_id.apply(
    lambda x: int(x.split("_")[2])
)
train_soundscape_labels["bird_call"] = train_soundscape_labels.birds.apply(
    lambda x: False if x == "nocall" else True
)
train_soundscape_labels["birds"] = train_soundscape_labels.birds.apply(
    lambda x: "" if x == "nocall" else x
)
train_soundscape_labels["birds"] = train_soundscape_labels.birds.apply(
    lambda x: json.dumps(split(x, " "))
)
train_soundscape_labels["bird_count"] = train_soundscape_labels.birds.apply(
    lambda x: len(eval(x))
)
train_soundscape_labels["birds"] = train_soundscape_labels.birds.apply(
    lambda x: eval(x)
)
train_soundscape_labels_org = train_soundscape_labels.copy()

# Add filename to metadata

In [ ]:
df = train_soundscape_labels_org.copy()
df["key"] = df.recording_id.astype(str) + "_" + df.site
df["file_glob"] = df.key + "_*.ogg"

siteaudio2filename = {}

for file_glob in df.file_glob.unique():
    audio_path = glob(f"../input/birdclef-2021/train_soundscapes/{file_glob}")[0]
    siteaudio2filename[file_glob[:-6]] = basename(audio_path)

df["filename"] = df.key.apply(lambda key: siteaudio2filename[key])
df = df.drop(["file_glob"], axis=1)

train_soundscape_labels = df.copy()
train_soundscape_labels.to_csv("train_soundscape_labels_v2.csv", index=False)
pd.read_csv("train_soundscape_labels_v2.csv")

# Speies appeared in each soundscapes

In [ ]:
data = {
    "recording_id": [],
    "species": [],
    "site": [],
}
grp = train_soundscape_labels.groupby("recording_id")
for i, (recording_id, df) in enumerate(grp):
    species = list(set(df.birds.sum()))
    data["recording_id"].append(recording_id)
    data["species"].append(species)
    data["site"].append(df.site.max())
pd.DataFrame(data).sort_values("site")

# Heat map of bird call

In [ ]:
class cfg:
    sample_rate = 32_000
    n_fft = 2048
    hop_length = 512
    n_mels = 128
    fmin = 0
    fmax = 16_000
    random_seed = -1

In [ ]:
def audio2img(audio_path, cfg, compression="pcen"):
    audio, _ = librosa.core.load(audio_path, sr=cfg.sample_rate, mono=True)
    spec = librosa.feature.melspectrogram(
        y=audio,
        sr=cfg.sample_rate,
        n_fft=cfg.n_fft,
        hop_length=cfg.hop_length,
        n_mels=cfg.n_mels,
        fmin=cfg.fmin,
        fmax=cfg.fmax,
        power=2,
    )
    if compression == "pcen":
        spec = librosa.pcen(
            spec * (2**31),
            time_constant=0.06,
            eps=1e-6,
            gain=0.8,
            power=0.25,
            bias=10,
            sr=cfg.sample_rate,
            hop_length=cfg.hop_length,
        )
    elif compression == "log":
        spec = librosa.power_to_db(spec, ref=np.max)

    return spec


def show_spec(spec, cfg, fig, ax):
    mesh = librosa.display.specshow(
        spec,
        hop_length=cfg.hop_length,
        sr=cfg.sample_rate,
        fmin=cfg.fmin,
        fmax=cfg.fmax,
        x_axis="time",
        y_axis="mel",
        ax=ax,
    )
    fig.colorbar(mesh, ax=ax, format="%+2.0f dB", location="right")


def normalize_spec(spec, factor=1.5):
    mean, std = spec.mean(), spec.std()
    spec = spec.clip(mean - std * factor, mean + std * factor)
    mean, std = spec.mean(), spec.std()
    spec = (spec - mean) / std
    return spec


def plot_spec(audio_path, cfg, g):
    spec = audio2img(audio_path, cfg)
    spec = normalize_spec(spec)

    g.imshow(
        spec[::-1, ...],  # frequency is reversed order
        aspect=g.get_aspect(),
        extent=g.get_xlim() + g.get_ylim(),
        zorder=1,
        cmap="magma",
    )

In [ ]:
def plot_heatmap(df, ax):
    mlb = MultiLabelBinarizer()
    if df.bird_count.sum() != 0:
        hmap = pd.DataFrame(mlb.fit_transform(df.birds), columns=mlb.classes_).T
    else:
        hmap = pd.DataFrame({"no call": [0] * 120}).T
    g = sns.heatmap(
        hmap,
        cbar=False,
        ax=ax,
        cmap="plasma",
        linecolor="gray",
        lw=1,
        zorder=2,
        alpha=0.2,
    )
    return g


def plot_heatmap_of_soundscape(df):
    df = df.copy()
    grp = df.groupby("recording_id")
    n_soundscapes = len(grp)
    fig, axes = plt.subplots(
        n_soundscapes, 1, figsize=(16, n_soundscapes * 2), sharex=True, sharey=False
    )
    for i, (recording_id, df) in enumerate(grp):
        ax = axes[i]
        site = df.site.max()
        audio_path = glob(
            f"../input/birdclef-2021/train_soundscapes/{recording_id}_{site}_*"
        )[0]

        g = plot_heatmap(df, ax)
        plot_spec(audio_path, cfg, g)

        ax.set(title=f"site:{site}, recording_id:{recording_id}", xticks=[])
    plt.tight_layout()

## Separate soundscape per site

In [ ]:
df = train_soundscape_labels.copy()
sites = df.site.unique()
site_dfs = [df.query("site == @site") for site in sites]

In [ ]:
plot_heatmap_of_soundscape(site_dfs[0])

In [ ]:
plot_heatmap_of_soundscape(site_dfs[1])

In [ ]:
unique_species = set()
for item in train_soundscape_labels.itertuples():
    for s in item.birds:
        unique_species.add(s)

print(f"total: {len(unique_species)} species")
print(f"species: [{', '.join(unique_species)}]")

In [ ]:
sns.countplot(x="site", data=train_soundscape_labels)
Counter(train_soundscape_labels.site)

In [ ]:
sns.countplot(x="bird_call", data=train_soundscape_labels)
Counter(train_soundscape_labels.bird_call)

In [ ]:
sns.countplot(x="bird_count", data=train_soundscape_labels)
Counter(train_soundscape_labels.bird_count)